#Part 1.3


##Librairies to import

In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
from sklearn.cluster import KMeans
from sklearn import metrics

##Clustering (text)

This part makes use of the previously stored (Part 1.2) text data of the file "preproc_texts.json", and presupposes that this file has been stored in the repository you are currently working in.

In [4]:
with open ("./preproc_texts.json", "r") as f :
  texts = json.load(f)
texts

{'Female_mathematicians': ["Éva Tardos (born 1 October 1957) is a Hungarian mathematician and the Jacob Gould Schurman Professor of Computer Science at Cornell University.\nTardos's research interest is algorithms. Her work focuses on the design and analysis of efficient methods for combinatorial optimization problems on graphs or networks. She has done some work on network flow algorithms like approximation algorithms for network flows, cut, and clustering problems. Her recent work focuses on algorithmic game theory and simple auctions.\n\n\n\nTardos received her Dipl. Math in 1981 and her Ph.D. 1984 from the Faculty of Sciences of the Eötvös Loránd University under her advisor András Frank. She was the Chair of the Department of Computer Science at Cornell from 2006-2010, and she is currently serving as the Associate Dean of the College of Computing and Information Science.\nShe was editor-in-Chief of SIAM Journal on Computing from 2004–2009, and is currently the Economics and Comput

In [5]:
#As our clustering corpus, we use the 200 biographies collected in Part 1.1

corpus = texts["Female_mathematicians"] + texts["Contemporary_artists"]

In [6]:
#Defining the assignment of the two labels for each biography
labels = ["f_math" for i in range(len(texts["Female_mathematicians"]))] + ["c_art" for i in range(len(texts["Contemporary_artists"]))]

df = pd.DataFrame({"corpus": corpus, "labels": labels})
df = df.sample(frac = 1) #shuffling the corpus
df

corpus  labels
114  Gottfried Helnwein (born 8 October 1948) is an...   c_art
195  INO is a visual artist from Greece who studied...   c_art
106  Vito Acconci (Italian: [ˈviːto akˈkontʃi], ; J...   c_art
188  Sir Gordon Howard Eliott Hodgkin  (6 August 19...   c_art
9    Márta Svéd (1910 – 30 September 2005) was a Hu...  f_math
..                                                 ...     ...
158  Maurizio Bolognini (born July 27, 1952) is a p...   c_art
36   Alina Carmen Cojocaru is a Romanian mathematic...  f_math
0    Éva Tardos (born 1 October 1957) is a Hungaria...  f_math
92   Eva Miranda Galcerán is a Spanish mathematicia...  f_math
193  Albert Oehlen (born 17 September 1954) is a Ge...   c_art

[200 rows x 2 columns]

In [7]:
#Setting the vectorizer
#We set the vectorizer so that stopwords are identified and not considered. Additionally, the vectorizer does lowercasing by default and we set an nltk tokenizer.
#This process is realised with a number of 200 features.

vectorizer = TfidfVectorizer(max_features=200, use_idf=True, stop_words='english', tokenizer=nltk.word_tokenize)

#Applying the vectorizer to the texts
M = vectorizer.fit_transform(df["corpus"])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
print(M.shape)

(200, 200)


In [9]:
#Getting the features
vectorizer.get_feature_names_out()

array(['$', '&', "'", "''", "'s", '(', ')', ',', '-', '.', '1', '1999',
       '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007',
       '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017', '2018', '2019', ':', ';', '``', 'academy', 'album',
       'american', 'andy', 'angeles', 'art', 'artist', 'artists', 'arts',
       'award', 'awarded', 'based', 'began', 'biennale', 'black', 'book',
       'born', 'bradford', 'british', 'california', 'called', 'career',
       'center', 'centre', 'chicago', 'city', 'collection', 'college',
       'contemporary', 'created', 'david', 'death', 'degree', 'design',
       'did', 'drawings', 'early', 'education', 'exhibited', 'exhibition',
       'exhibitions', 'family', 'father', 'fellow', 'film', 'fine',
       'foundation', 'gallery', 'germany', 'glass', 'group', 'held',
       'high', 'history', 'images', 'include', 'included', 'including',
       'installation', 'institute', 'international', 'isbn', 'ja

In [10]:
#Fitting the model to the data

km2 = KMeans(n_clusters = 2, n_init = 100, random_state = 10)
km2.fit(M)

KMeans(n_clusters=2, n_init=100, random_state=10)

In [11]:
predicted_labels = km2.labels_.tolist()

In [12]:
df['labels'] = (df['labels'] == 'c_art').astype(int)
df.insert(2, "pred_labels", predicted_labels)

In [13]:
df #showing the text data associated with the original labels and the ones predicted by the clustering model

corpus  labels  pred_labels
114  Gottfried Helnwein (born 8 October 1948) is an...       1            1
195  INO is a visual artist from Greece who studied...       1            1
106  Vito Acconci (Italian: [ˈviːto akˈkontʃi], ; J...       1            1
188  Sir Gordon Howard Eliott Hodgkin  (6 August 19...       1            1
9    Márta Svéd (1910 – 30 September 2005) was a Hu...       0            0
..                                                 ...     ...          ...
158  Maurizio Bolognini (born July 27, 1952) is a p...       1            1
36   Alina Carmen Cojocaru is a Romanian mathematic...       0            0
0    Éva Tardos (born 1 October 1957) is a Hungaria...       0            0
92   Eva Miranda Galcerán is a Spanish mathematicia...       0            0
193  Albert Oehlen (born 17 September 1954) is a Ge...       1            1

[200 rows x 3 columns]

In [14]:
#Getting the number of texts that were predicted to be of each of the two labels

df['pred_labels'].value_counts()

pred_labels
1    106
0     94
Name: count, dtype: int64

We can observe that the results are a bit far from the fifty-fifty expectation.

In [15]:
#Metrics to measure the efficiency of the clustering

print("Homogeneity: %0.3f" % metrics.homogeneity_score(df["labels"], km2.labels_))
print("Completeness: %0.3f" % metrics.completeness_score(df["labels"], km2.labels_))
print("V-measure: %0.3f" % metrics.v_measure_score(df["labels"], km2.labels_))
print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(df["labels"], km2.labels_))

Homogeneity: 0.834
Completeness: 0.836
V-measure: 0.835
Adjusted Rand-Index: 0.883


In [16]:
print("Top terms per cluster:")

# get the cluster center of each cluster
# argsort() return the index of each dimension in the cluster center and sort them in increasing value order
# [:, ::-1] reverts the argsort() list to place the indices with highest value first (decreasing order)
order_centroids = km2.cluster_centers_.argsort()[:]

# terms maps a vectorizer index to the corresponding token
terms = vectorizer.get_feature_names_out()

# for each cluster
for i in range(2):
    print("Cluster %d:" % i, end='')
    # print out the token of the centroid (order by decreasing tf-idf value)
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind], end='')
    print('\n')

Top terms per cluster:
Cluster 0: album retrospective images perry koons paintings painting sculpture paik objects

Cluster 1: theory mathematician science lennon fellow mathematical woman research education 2018

